In [ ]:
import numpy as np
import pywt
import equimagelab as eqlab
dbrd = eqlab.Dashboard(port = 8050, debug = True) # Start the dashboard.

# Load image and convert to B&W:

In [ ]:
image, meta = eqlab.load_image("M45.png")

In [ ]:
bwimage = image.grayscale(channel = "L*")

In [ ]:
eqlab.show(bwimage)
dbrd.show(bwimage, histograms = True, statistics = True)

# Test wavelets transform:

In [ ]:
wt = bwimage.slt(starlet = "cubic", levels = 6, mode = "reflect")

In [ ]:
dbrd.show_wavelets(wt, absc = True, normalize = True)

In [ ]:
backward = wt.iwt()

In [ ]:
diff = abs(backward-bwimage)
print(np.max(diff))

# Additive gaussian noise:

In [ ]:
noise = 0.033
noisy = bwimage+noise*np.random.normal(size = bwimage.get_shape())

In [ ]:
eqlab.show(noisy)
dbrd.show((noisy, bwimage), histograms = True, statistics = True)

In [ ]:
wt = noisy.slt(starlet = "cubic", levels = 6, mode = "reflect")

In [ ]:
clip = wt.VisuShrink_clip()
clip = 3.
print(f"Clip = {clip}.")

In [ ]:
estnoise = wt.estimate_noise(clip = clip)[1][0]
print(f"Estimated noise = {estnoise}.")

In [ ]:
denoised1, diff1 = wt.iterative_noise_reduction(clip = clip, eps = 1.e-6, maxit = 0)

In [ ]:
denoised, diff = wt.iterative_noise_reduction(clip = clip, eps = 1.e-6, maxit = 24)

In [ ]:
eqlab.show(denoised)
dbrd.show({"Original": bwimage, "Noisy": noisy, "Single": denoised1, "Iterative": denoised, "Single diff": 7.*abs(diff1), "Iterative diff": 7.*abs(diff)}, histograms = True, statistics = True)

# Poisson noise:

In [ ]:
poisson = 33
noisy = eqlab.Image(np.random.poisson(poisson*bwimage))/poisson

In [ ]:
eqlab.show(noisy)
dbrd.show((noisy, bwimage), histograms = True, statistics = True)

### Without Anscombe transform:

In [ ]:
wt = noisy.slt(starlet = "cubic", levels = 6, mode = "reflect")

In [ ]:
clip = wt.VisuShrink_clip()
clip = 3.
print(f"Clip = {clip}.")

In [ ]:
estnoise = wt.estimate_noise(clip = clip)[1][0]
print(f"Estimated noise = {estnoise}.")

In [ ]:
denoised1, diff1 = wt.iterative_noise_reduction(clip = clip, eps = 1.e-6, maxit = 0)

In [ ]:
denoised, diff = wt.iterative_noise_reduction(clip = clip, eps = 1.e-6, maxit = 24)

In [ ]:
eqlab.show(denoised)
dbrd.show({"Original": bwimage, "Noisy": noisy, "Single": denoised1, "Iterative": denoised, "Single diff": 4.*abs(diff1), "Iterative diff": 4.*abs(diff)}, histograms = True, statistics = True)

In [ ]:
gdenoised, gdiff = denoised, diff

### With Anscombe transform:

In [ ]:
gAt = noisy.anscombe(gain = 1./poisson)

In [ ]:
wt = gAt.slt(starlet = "cubic", levels = 6, mode = "reflect")

In [ ]:
clip = wt.VisuShrink_clip()
clip = 3.
print(f"Clip = {clip}.")

In [ ]:
estnoise = wt.estimate_noise(clip = clip)[1][0]
print(f"Estimated noise = {estnoise}.")

In [ ]:
denoisedgAt1, diff1 = wt.iterative_noise_reduction(clip = clip, eps = 1.e-6, maxit = 0)

In [ ]:
denoisedgAt, diff = wt.iterative_noise_reduction(clip = clip, eps = 1.e-6, maxit = 24)

In [ ]:
denoised1 = denoisedgAt1.inverse_anscombe(gain = 1./poisson)
denoised =  denoisedgAt.inverse_anscombe(gain = 1./poisson)

In [ ]:
eqlab.show(denoised)
dbrd.show({"Original": bwimage, "Noisy": noisy, "Single": denoised1, "Iterative": denoised, "Single diff": abs(diff1)/4., "Iterative diff":  abs(diff)/4.}, histograms = True, statistics = True)

### Compare with/without Anscombe transform:

In [ ]:
dbrd.show({"Original": bwimage, "Noisy": noisy, "With Anscombe": denoised, "Without Ascombe": gdenoised}, histograms = True, statistics = True)

### With extra hot & cold pixels correction:

In [ ]:
gdenoisedhcp = gdenoised.remove_hot_cold_pixels(hot_ratio = 1.1, cold_ratio = 1.1)
denoisedhcp = denoised.remove_hot_cold_pixels(hot_ratio = 1.1, cold_ratio = 1.1)

In [ ]:
dbrd.show({"Original": bwimage, "Noisy": noisy, "With Anscombe": denoisedhcp, "Without Ascombe": gdenoisedhcp}, histograms = True, statistics = True)